In [71]:
import regex as re
import pandas as pd
from textblob import TextBlob

In [72]:
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    # splitted_tweet = tweet.split()
    cleaned = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])"
                           "|(\w+:\/\/\S+)|(#[A-Za-z0-9]+)", " ", tweet)
    cleaned_no_links = []
    for word in cleaned.split():
        if word.startswith("https?://\S+"):
            cleaned_no_links.append("url")
        else:
            cleaned_no_links.append(word)

    return ' '.join(cleaned_no_links)

def get_tweet_sentiment(tweet):
    # create TextBlob object of tweet text
    analysis = TextBlob(clean_tweet(tweet))
    return analysis.sentiment

In [73]:
df = pd.read_csv("files/brooklyn99.csv")

In [74]:
for x in range(400):
    tweet = df["text"].iloc[x]
    clean = clean_tweet(tweet)
    print(clean)
    print(get_tweet_sentiment(clean))

She was great on
Sentiment(polarity=0.8, subjectivity=0.75)
Watch brooklyn nine nine
Sentiment(polarity=0.0, subjectivity=0.0)
Westbrook had nine turnovers last night that shit is not ok Brooklyn has my pick this year so far
Sentiment(polarity=-0.0875, subjectivity=0.5916666666666667)
33 Brooklyn nine nine squid games the office
Sentiment(polarity=0.0, subjectivity=0.0)
How can Brooklyn Nine Nine make me cry at 8 am on a Monday Lemme go do the ppl dem work
Sentiment(polarity=0.0, subjectivity=0.0)
this is the quiz of the day
Sentiment(polarity=0.0, subjectivity=0.0)
I m watching Brooklyn Nine Nine to neutralize the effect of the House of Secrets
Sentiment(polarity=0.0, subjectivity=0.0)
Brooklyn nine nine
Sentiment(polarity=0.0, subjectivity=0.0)
New Girl Brooklyn Nine Nine Queen s Gambit Hannibal Superstore The Mandalorian 3 3
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453)
Done watching Brooklyn Nine Nine bittersweet amp
Sentiment(polarity=0.0, subjectivity=

IndexError: single positional indexer is out-of-bounds